## Question 1. 

* Tensorflow is open-source library for numerical computation.  
* It particularly made for large scale machine learning task which support GPU computation.  
* Optimization API based on reverse-mode autofidd which is fast to compute gradient and Jacobian.


## Question 2. 

It is not a drop-in replacement, as it have some differences in naming, computing and characteristics.

## Question 3

They are different in datatype, tensorflow default int is int32 while numpy default int is int64.

## Question 4

* Tensor
* Tensor Variable. 
* Tensor Array. 
* Tensor String. 
* 

## Question 5

* Custom loss function should be used if there are no hyperparameters in the function. As it will not be saved along with the model. 
* And if you use the subclass method, you have to override get_config() method to saved neccessary hyperparmeter.

## Question 6

* If the custom metric have to compute something that doesn't equal between the whole epoch and the mean metric over all the batches. You have to use sublass.

## Question 7

* Custom layers should be used for the reusable blocks that cannot train by itselves.  
* Custom models should be used for trainable objects.

## Question 8

Custom training loops should not be used, if it is not absoultely needed. Because it is error prone and the code will be longer. However, if we want to implement some complex models that comes out from research paper, we have to use custom training loops. We have to  
* create our model  
* create gradient tape for cost function  
* forward the input  
* get the gradient  
* update the trainable parameters  
* print the status  
* repeat

## Question 9

* Keras custom component should stick to TF operations as much as possible to improve performance and portable to another devices.  
* You can use arbitrary python code though, by setting decorating with tf.py_function() or by passing argument dynamic=True in custom layers.

## Question 10

## Question 11

* Dynamic model prevent keras to build a graph. This create limitation in portability. Dynamic model only executable if the machine install python.  
* Also, the performance is also lower because the pure python code cannot be optimize into a graph.

## Question 12

In [24]:
## create cutom layers that perform layernormalization
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
import numpy as np

In [204]:
class MyLayerNorm(Layer):
    '''
    init
    build
    call
    
    '''
    def __init__(self, axis = 0, eps = 0.001, *args, **kwargs):
        self.axis = 0
        self.eps = eps
        super().__init__(*args, **kwargs)
        return
    
    def build(self, input_shape):
        self.beta = tf.Variable(tf.zeros(input_shape[-1:]))
        self.alpha = tf.Variable(tf.ones(input_shape[-1:]))
        
    def call(self, inputs):
        mean, variance = tf.nn.moments(inputs, axes = 1, keepdims=True,)
        print(f'input : {inputs}')
        print(f'mean : {mean.shape}')
        print(f'variance : {variance.shape}')
        print(f'beta : {self.beta.shape}')
        print(f'alpha : {self.alpha.shape}')
        outputs = (inputs-mean)/(tf.sqrt(variance) + self.eps)
        # shift and rescale
        outputs = outputs*self.alpha + self.beta
        return outputs

In [226]:
arr = np.arange(20).reshape(2,10).astype('float32')
print(arr.dtype)
print(arr.shape)

float32
(2, 10)


In [227]:
tf.keras.backend.clear_session()

l1 = LayerNormalization()

In [228]:
l2 = MyLayerNorm()

In [231]:
l1(arr)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-1.566604  , -1.2184697 , -0.8703356 , -0.5222013 , -0.17406711,
         0.17406711,  0.5222013 ,  0.8703356 ,  1.2184697 ,  1.566604  ],
       [-1.566604  , -1.2184697 , -0.8703356 , -0.5222013 , -0.17406711,
         0.17406711,  0.5222013 ,  0.8703356 ,  1.2184697 ,  1.566604  ]],
      dtype=float32)>

In [230]:
l2(arr)

input : [[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]]
mean : (2, 1)
variance : (2, 1)
beta : (10,)
alpha : (10,)


<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-1.5661536 , -1.2181195 , -0.87008536, -0.5220512 , -0.17401707,
         0.17401707,  0.5220512 ,  0.87008536,  1.2181195 ,  1.5661536 ],
       [-1.5661536 , -1.2181195 , -0.87008536, -0.5220512 , -0.17401707,
         0.17401707,  0.5220512 ,  0.87008536,  1.2181195 ,  1.5661536 ]],
      dtype=float32)>

## Question 13. 

In [310]:
## create custom training loops
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import sparse_categorical_crossentropy, Accuracy, Mean
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets as tfds

In [234]:
(X_train, y_train), (X_test, y_test) = tfds.fashion_mnist.load_data()

In [235]:
print(X_train.shape)

(60000, 28, 28)


In [402]:
X_train_scale = X_train/255.
X_test_scale = X_test/255.

In [422]:
def print_status(epoch, max_step, step, metrics, metrics_name, loss):
    
    status = ' - '.join(['{} {:.3f}'.format(metrics_name[i], float(m_result.result())) for i, m_result in enumerate([loss] + metrics)])
    end = '' if step < max_step else '\n'
    
    print('\repoch {} {} {}/{}'.format(epoch, status, step, max_step), end=end)

In [425]:
## define loss, custom metrics
mean_loss = Mean()
metrics = [Accuracy()]
metrics_name = ['loss', 'acc']

epochs = 20
batch_size = 32
max_step = X_train.shape[0]

optim1 = tf.keras.optimizers.Adam()
optim2 = tf.keras.optimizers.SGD(lr = 1e-4, momentum=0.9, nesterov=True)

In [420]:
model = Sequential([
    Flatten(input_shape = [28, 28]),
    Dense(100, activation = 'elu'),
    Dense(100, activation = 'elu'),
    Dense(10, activation = 'softmax')
])

In [424]:
np.random.seed(42)
tf.keras.backend.clear_session()

for epoch in range(1, epochs+1):
    
    for step in range(X_train.shape[0]//batch_size+1):
        batch_idx = np.random.choice(60000,size = batch_size)
        X_batch, y_batch = X_train_scale[batch_idx], y_train[batch_idx]
        
        with tf.GradientTape() as tape:
            output = model(X_batch)
            main_loss = tf.reduce_mean(sparse_categorical_crossentropy(y_batch, output))
            loss = tf.add_n([main_loss], model.losses)
        
        gradient = tape.gradient(loss, model.trainable_variables)
        
#         optim1.apply_gradients(zip(gradient[3:], model.trainable_variables[3:]))
#         optim2.apply_gradients(zip(gradient[:3], model.trainable_variables[:3]))
        optim2.apply_gradients(zip(gradient, model.trainable_variables))
        
        mean_loss(loss)
        
        for metric in metrics:
            metric(y_batch, tf.argmax(output, axis = 1))
        
        print_status(epoch, max_step, step*batch_size, metrics, metrics_name, mean_loss)

    if epoch == 2:
        break

epoch 1 loss 1.357 - acc 0.570 60000/60000
epoch 2 loss 1.087 - acc 0.651 60000/60000


In [445]:
metrics[0](y_test, np.argmax(model.predict(X_test_scale), axis=-1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.71234375>

## Train on adaptive optim

In [431]:
model2 = Sequential([
    Flatten(input_shape = [28, 28]),
    Dense(100, activation = 'elu'),
    Dense(100, activation = 'elu'),
    Dense(10, activation = 'softmax')
])

In [433]:
np.random.seed(42)
tf.keras.backend.clear_session()

for epoch in range(1, epochs+1):
    
    for step in range(X_train.shape[0]//batch_size+1):
        batch_idx = np.random.choice(60000,size = batch_size)
        X_batch, y_batch = X_train_scale[batch_idx], y_train[batch_idx]
        
        with tf.GradientTape() as tape:
            output = model2(X_batch)
            main_loss = tf.reduce_mean(sparse_categorical_crossentropy(y_batch, output))
            loss = tf.add_n([main_loss], model2.losses)
        
        gradient = tape.gradient(loss, model2.trainable_variables)
        
#         optim1.apply_gradients(zip(gradient[3:], model.trainable_variables[3:]))
#         optim2.apply_gradients(zip(gradient[:3], model.trainable_variables[:3]))
        optim1.apply_gradients(zip(gradient, model2.trainable_variables))
        
        mean_loss(loss)
        
        for metric in metrics:
            metric(y_batch, tf.argmax(output, axis = 1))
        
        print_status(epoch, max_step, step*batch_size, metrics, metrics_name, mean_loss)

    if epoch == 2:
        break

epoch 1 loss 0.470 - acc 0.832 60000/60000
epoch 2 loss 0.419 - acc 0.849 60000/60000


In [444]:
metrics[0](y_test, np.argmax(model2.predict(X_test_scale), axis=-1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.72956306>

## Train top level with adaptive and train low level with momentum SGD

In [436]:
model3 = Sequential([
    Flatten(input_shape = [28, 28]),
    Dense(100, activation = 'elu'),
    Dense(100, activation = 'elu'),
    Dense(10, activation = 'softmax')
])

In [437]:
np.random.seed(42)
tf.keras.backend.clear_session()

for epoch in range(1, epochs+1):
    
    for step in range(X_train.shape[0]//batch_size+1):
        batch_idx = np.random.choice(60000,size = batch_size)
        X_batch, y_batch = X_train_scale[batch_idx], y_train[batch_idx]
        
        with tf.GradientTape() as tape:
            output = model3(X_batch)
            main_loss = tf.reduce_mean(sparse_categorical_crossentropy(y_batch, output))
            loss = tf.add_n([main_loss], model3.losses)
        
        gradient = tape.gradient(loss, model3.trainable_variables)
        
        optim1.apply_gradients(zip(gradient[3:], model3.trainable_variables[3:]))
        optim2.apply_gradients(zip(gradient[:3], model3.trainable_variables[:3]))
        
        mean_loss(loss)
        
        for metric in metrics:
            metric(y_batch, tf.argmax(output, axis = 1))
        
        print_status(epoch, max_step, step*batch_size, metrics, metrics_name, mean_loss)

    if epoch == 2:
        break

epoch 1 loss 0.831 - acc 0.697 60000/60000
epoch 2 loss 0.765 - acc 0.722 60000/60000


In [439]:
metrics[0](y_test, tf.argmax(model3(X_test_scale), axis = 1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.72514975>

**It seems that the adam optimizer has the most converge rate**. 
However, the generalize error seems to be equal with SGD optimizer.  
While, the model which train with adam and SGD performance and convergence rate is between Adam and SGD!